<a href="https://colab.research.google.com/github/abood-W/ai_python/blob/main/IMDB_movie_rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()
df['sentiment'] = df['sentiment'].str.strip().str.lower()
df['sentiment'] = (df['sentiment'] == 'positive').astype(int)

import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)
df['clean_review'] = df['review'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['clean_review'])
sequences = tokenizer.texts_to_sequences(df['clean_review'])
X = pad_sequences(sequences,maxlen=100, padding='post')

In [23]:
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
model = keras.Sequential([
    keras.layers.Embedding(10000,16),
    keras.layers.Bidirectional(keras.layers.LSTM(20)),
    keras.layers.Dense(1,activation='sigmoid')
])

In [25]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [41]:
model.fit(X_train, y_train, epochs=5, batch_size=512, validation_split=0.2)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 201ms/step - accuracy: 0.9798 - loss: 0.0794 - val_accuracy: 0.8671 - val_loss: 0.4631
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 13s 200ms/step - accuracy: 0.9826 - loss: 0.0741 - val_accuracy: 0.8643 - val_loss: 0.5018
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 201ms/step - accuracy: 0.9805 - loss: 0.0807 - val_accuracy: 0.8635 - val_loss: 0.4907
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 200ms/step - accuracy: 0.9850 - loss: 0.0652 - val_accuracy: 0.8633 - val_loss: 0.5095
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 198ms/step - accuracy: 0.9853 - loss: 0.0632 - val_accuracy: 0.8644 - val_loss: 0.5158


In [42]:
prediction = model.predict(X_test)
print("prediction: " , prediction)

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step
prediction:  [[0.9886427 ]
 [0.9942629 ]
 [0.00270248]
 ...
 [0.98977256]
 [0.97994256]
 [0.9779562 ]]


In [43]:
from sklearn.metrics import accuracy_score
predict_lable = (prediction > 0.5).astype(int)
accuracy = accuracy_score(y_test, predict_lable)
print("accuracy: " , accuracy*100,"%")

accuracy:  85.98 %
